In [5]:
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import MetaData
import getpass
password = getpass.getpass()
from sqlalchemy_utils import database_exists, create_database
import pandas as pd

In [6]:
def date_formatting (df, col):
    df[col] = df[col].astype("datetime64[ns]")
    df["year"] = pd.DatetimeIndex(df[col]).year
    df = df.sort_values(by=col, ascending = 0)
    df = df.drop_duplicates(subset = [col])
    return df

def snake_case(df):
    cols=[]
    for i in df.columns:
        cols.append(i.lower())
    df.columns = cols
    df.columns = df.columns.str.replace(' ', '_')
    return df

In [77]:
gold = pd.read_csv("gold price.csv")
silver = pd.read_csv("silver price.csv")
dji = pd.read_csv("dow jones industrial stock price.csv")
djt = pd.read_csv("dow jones transportation stock price.csv")
snp = pd.read_csv("snp stock price.csv")
platinum = pd.read_csv("platinum price.csv")
oil = pd.read_csv("oil price.csv")
palladium = pd.read_csv("palladium price.csv")
eur = pd.read_csv("eur price.csv")
ngas = pd.read_csv("natural gas price.csv")
dax = pd.read_csv("dax transportation & logistics price.csv")
coal = pd.read_csv("coal price.csv")
nasdaq = pd.read_csv("nasdaq price.csv")
gbp = pd.read_csv("gbp price.csv")
copper = pd.read_csv("southern copper stock price.csv")
rio = pd.read_csv("rio tinto stock price.csv")
rgold = pd.read_csv("royal gold price.csv")
agnico = pd.read_csv("agnico mines price.csv")
newmont = pd.read_csv("newmont stock price.csv")


In [78]:
nasdaq = nasdaq[["Date","Close"]]
nasdaq = nasdaq.rename(columns = {"Close":"Price"})
rgold = rgold[["Date","Close"]]
rgold = rgold.rename(columns = {"Close":"Price"})
agnico = agnico[["Date","Close"]]
agnico = agnico.rename(columns = {"Close":"Price"})
platinum = platinum[["Date","Price"]]
oil = oil[["Date","Price"]]
palladium = palladium[["Date","Price"]]
eur = eur[["Date","Price"]]
ngas = ngas[["Date","Price"]]
dax = dax[["Date","Price"]]
coal = coal[["Date","Price"]]
gbp = gbp[["Date","Price"]]
# if this cell makes an error, try running 1 previous cell, and then run this one.

In [80]:
# dp = [platinum,oil,palladium,eur,ngas,dax,coal,gbp]
# dc = [nasdaq,rgold,agnico]
df_all = [gold,silver,dji,djt,snp,platinum,oil,palladium,eur,ngas,dax,coal,nasdaq,gbp,copper,rio,rgold,agnico,newmont]
for i in df_all:
    i = snake_case(i)
    i = date_formatting(i, "date")

In [81]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/price_predict'
engine = create_engine(connection_string)
if not database_exists(engine.url):
    create_database(engine.url)

In [82]:
# gold,silver,dji,djt,snp,platinum,oil,palladium,eur,ngas,dax,coal,nasdaq,gbp,copper,rio,rgold,agnico,newmont

gold.to_sql("gold", engine, if_exists = "replace")
silver.to_sql("silver", engine, if_exists = "replace")
dji.to_sql("dji", engine, if_exists = "replace")
djt.to_sql("djt", engine, if_exists = "replace")
snp.to_sql("snp", engine, if_exists = "replace")
platinum.to_sql("platinum", engine, if_exists = "replace")
oil.to_sql("oil", engine, if_exists = "replace")
palladium.to_sql("palladium", engine, if_exists = "replace")
eur.to_sql("eur", engine, if_exists = "replace")
ngas.to_sql("ngas", engine, if_exists = "replace")
dax.to_sql("dax", engine, if_exists = "replace")
coal.to_sql("coal", engine, if_exists = "replace")
nasdaq.to_sql("nasdaq", engine, if_exists = "replace")
gbp.to_sql("gbp", engine, if_exists = "replace")
copper.to_sql("copper", engine, if_exists = "replace")
rio.to_sql("rio", engine, if_exists = "replace")
rgold.to_sql("rgold", engine, if_exists = "replace")
agnico.to_sql("agnico", engine, if_exists = "replace")
newmont.to_sql("newmont", engine, if_exists = "replace")

1791

In [85]:
query = '''
select gold.date, gold.year, gold.price as gold,
agnico.price as agnico, coal.price as coal,
copper.price as copper, dax.price as dax,
dji.price as dji, djt.price as djt, eur.price as eur,
gbp.price as gbp, nasdaq.price as nasdaq,
newmont.price as newmont, ngas.price as n_gas,
oil.price as oil, palladium.price as palladium,
platinum.price as platinum, rgold.price as rgold,
rio.price as rio, silver.price as silver, snp.price as snp
from gold left join agnico on gold.date = agnico.date
left join coal on gold.date = coal.date
left join copper on gold.date = copper.date
left join dax on gold.date = dax.date
left join dji on gold.date = dji.date
left join djt on gold.date = djt.date
left join eur on gold.date = eur.date
left join gbp on gold.date = gbp.date
left join nasdaq on gold.date = nasdaq.date
left join newmont on gold.date = newmont.date
left join ngas on gold.date = ngas.date
left join oil on gold.date = oil.date
left join palladium on gold.date = palladium.date
left join platinum on gold.date = platinum.date
left join rgold on gold.date = rgold.date
left join rio on gold.date = rio.date
left join silver on gold.date = silver.date
left join snp on gold.date = snp.date
order by gold.date asc;;
'''
data = pd.read_sql_query(query, engine)
data.shape

(1895, 21)

In [86]:
data.to_csv("data_raw.csv")